In [1]:
import import_ipynb
import numpy as np
import coordinatecube as CoordC
import TwoPhaseCoordinate as TPCoordC
import time

importing Jupyter notebook from coordinatecube.ipynb
importing Jupyter notebook from cubiecube.ipynb
importing Jupyter notebook from corner.ipynb
importing Jupyter notebook from facelet.ipynb
importing Jupyter notebook from edge.ipynb
importing Jupyter notebook from facecube.ipynb
importing Jupyter notebook from color.ipynb
Cube in FaceCube level is a string: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB

We can convert it into cube-like string: 
    |UUU|
    |UUU|
    |UUU|
|LLL|FFF|RRR|BBB|
|LLL|FFF|RRR|BBB|
|LLL|FFF|RRR|BBB|
    |DDD|
    |DDD|
    |DDD|

Cube in CubieCube level is represented by cp, co, ep, eo:
cp = [0, 1, 2, 3, 4, 5, 6, 7]
co = [0 0 0 0 0 0 0 0]
ep = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
eo = [0 0 0 0 0 0 0 0 0 0 0 0]

We can translate it into FaceCube level and cube-like string: 
UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB 
    |UUU|
    |UUU|
    |UUU|
|LLL|FFF|RRR|BBB|
|LLL|FFF|RRR|BBB|
|LLL|FFF|RRR|BBB|
    |DDD|
    |DDD|
    |DDD|

afte

NameError: name 'cube_in_coordinate_level' is not defined

In [12]:
U, U2, U3, R, R2, R3, F, F2, F3, D, D2, D3, L, L2, L3, B, B2, B3 = [i for i in range(18)]
move_dict = {
    'U' : U, 
    'UU' : U2, 
    'UUU' : U3, 
    'R' : R, 
    'RR' : R2, 
    'RRR' : R3, 
    'F' : F, 
    'FF' : F2, 
    'FFF' : F3, 
    'D' : D, 
    'DD' : D2, 
    'DDD' : D3, 
    'L' : L, 
    'LL' : L2, 
    'LLL' : L3, 
    'B' : B, 
    'BB' : B2, 
    'BBB' : B3
}

inv_move_index = {
    'U' : 'UUU', 
    'UU' : 'UU', 
    'UUU' : 'U', 
    'R' : 'RRR', 
    'RR' : 'RR', 
    'RRR' : 'R', 
    'F' : 'FFF', 
    'FF' : 'FF', 
    'FFF' : 'F', 
    'D' : 'DDD', 
    'DD' : 'DD', 
    'DDD' : 'D', 
    'L' : 'LLL', 
    'LL' : 'LL', 
    'LLL' : 'L', 
    'B' : 'BBB', 
    'BB' : 'BB', 
    'BBB' : 'B'
}

In [3]:
class MoveTable():
    def __init__(self):
        # row entries: different pattern
        #column entries: 18 moves: U, U2, U3, R, R2, R3, F, F2, F3, D, D2, D3, L, L2, L3, B, B1, B2, B3
        self.CornerPermutation = np.zeros([np.math.factorial(8),18], dtype = 'int')
        self.CornerOrientation = np.zeros([3**7,18],dtype = 'int')
        self.EdgePermutation_P1 = np.zeros([12*11*10*9//np.math.factorial(4),18], dtype = 'int') 
        self.EdgeOrientation = np.zeros([2**11,18],dtype = 'int')
           
        # Corner Permutation
        for i in range(np.math.factorial(8)):
            # generate a cube, turn it, record it, and then undo the move.
            Q = CoordC.CoordinateCube(i, 0, 0, 0, CubieCube = None)
            Q_C = Q.to_cubiecube()
            for key in move_dict.keys():
                Q_C.move(key)
                Q_2 = CoordC.CoordinateCube(CubieCube = Q_C)
                self.CornerPermutation[i,move_dict[key]] = Q_2.cp
                Q_C.move(inv_move_index[key])
        
        # Corner Orientation
        for i in range(3**7):
            # generate a cube, turn it, record it, and then undo the move.
            Q = CoordC.CoordinateCube(0, i, 0, 0, CubieCube = None)
            Q_C = Q.to_cubiecube()
            for key in move_dict.keys():
                Q_C.move(key)
                Q_2 = CoordC.CoordinateCube(CubieCube = Q_C)
                self.CornerOrientation[i,move_dict[key]] = Q_2.co
                Q_C.move(inv_move_index[key])
        
        # Edge Permutation
        for i in range(12*11*10*9//np.math.factorial(4)):
            # generate a cube, turn it, record it, and then undo the move.
            Q = TPCoordC.PhaseOneCoordinateCube(0, 0, i, 0, CubieCube = None)
            Q_C = Q.to_CubieCube()
            for key in move_dict.keys():
                Q_C.move(key)
                Q_2 = TPCoordC.PhaseOneCoordinateCube(CubieCube = Q_C)
                self.EdgePermutation_P1[i,move_dict[key]] = Q_2.ep_UDSlice_phase1
                Q_C.move(inv_move_index[key])
        
        # Edge Orientation
        for i in range(2**11):
            # generate a cube, turn it, record it, and then undo the move.
            Q = CoordC.CoordinateCube(0, 0, 0, i, CubieCube = None)
            Q_C = Q.to_cubiecube()
            for key in move_dict.keys():
                Q_C.move(key)
                Q_2 = CoordC.CoordinateCube(CubieCube = Q_C)
                self.EdgeOrientation[i,move_dict[key]] = Q_2.eo
                Q_C.move(inv_move_index[key])

In [33]:
start_time = time.time()
MT = MoveTable()
print("--- %s seconds ---" % (time.time() - start_time))

--- 271.0575575828552 seconds ---


In [27]:
np.savetxt('MoveTable_cp.txt', MT.CornerPermutation, fmt='%d')
np.savetxt('MoveTable_co.txt', MT.CornerOrientation, fmt='%d')
np.savetxt('MoveTable_ep_P1.txt', MT.EdgePermutation_P1, fmt='%d')
np.savetxt('MoveTable_eo.txt', MT.EdgeOrientation, fmt='%d')

In [5]:
MoveTable_cp = np.loadtxt('MoveTable_cp.txt', dtype=int)
MoveTable_co = np.loadtxt('MoveTable_co.txt', dtype=int)
MoveTable_ep_P1 = np.loadtxt('MoveTable_ep_P1.txt', dtype=int)
MoveTable_eo = np.loadtxt('MoveTable_eo.txt', dtype=int)

In [21]:
MoveTable_cp = np.array(MoveTable_cp)
MoveTable_co = np.array(MoveTable_co)
MoveTable_ep_P1 = np.array(MoveTable_ep_P1)
MoveTable_eo = np.array(MoveTable_eo)
print(f"the shape of four tables:\n\
cp: {MoveTable_cp.shape}\n\
co: {MoveTable_co.shape}\n\
ep_UDSlice_phase1: {MoveTable_ep_P1.shape}\n\
eo: {MoveTable_eo.shape}")


the shape of four tables:
cp: (40320, 18)
co: (2187, 18)
ep_UDSlice_phase1: (495, 18)
eo: (2048, 18)


In [23]:
print(MoveTable_co[:10, :])

[[   0    0    0 1494    0 1494 1236    0 1236    0    0    0  412    0
   412  137    0  137]
 [   1    1    1 1486 1459 1468 1237    1 1237    5   15   18  250  243
   406  110  189  217]
 [   2    2    2 1505  731 1523 1238    2 1238    7   21    9  331  486
   409   83  135   54]
 [   3    3    3 1488 1461 1470  750  729 1218   11    6   19  413   81
   655  113  165  139]
 [   4    4    4 1507  733 1525  751  730 1219   13   12   10  251  324
   649   86  111  219]
 [   5    5    5 1499    5 1499  752  731 1220   15   18    1  332  567
   652  140   57   59]
 [   6    6    6 1509  735 1527  993 1458 1227   19    3   11  411  162
   169   89   87  141]
 [   7    7    7 1501    7 1501  994 1459 1228   21    9    2  249  405
   163  143   33  224]
 [   8    8    8 1493 1466 1475  995 1460 1229   26   24   20  330  648
   166  116  222   61]
 [   9    9    9   27 1485 1467 1239  243 1965    2    7   21  421    9
   421  119  171  145]]


In [29]:
# the (258, U)-entry of the move table 
# of co is 96 means that a cube with 
# co = 258 converts to 96  after a U move.
print(MoveTable_co[258, U])

96
